In [1]:
import mariadb as mdb
import json
import pandas as pd
from sqlalchemy import create_engine
import pleiades as ple

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 300)

In [2]:
engine = create_engine('mysql+pymysql://root:@localhost/testDB')
con = engine.connect()

In [3]:
dbname = 'testDB'
cfg_path = '../server.cfg'

with open(cfg_path, 'r') as f:
    server_config = json.load(f)
db = mdb.connect(**server_config)
cursor = db.cursor()

In [4]:
command = f'USE {dbname};'
try:
    cursor.execute(command)
except mdb.ProgrammingError:
    print('database does not exist')

In [5]:
cz = ple.CZ(engine, alchemy=True)

In [6]:
cz.select_from('konosuba').ex()

,id,name,class,race,sex,isekai,age
0,1,Kazuma,adventurer,human,M,1,17.0
1,2,Aqua,priest,god,F,0,NaN
2,3,Megumin,wizard,human,F,0,14.0
3,4,Darkness,crusader,human,F,0,19.0
4,5,Chris,thief,human,F,0,15.0
5,6,Yunyun,wizard,human,F,0,14.0
6,7,Wiz,wizard,lich,F,0,20.0
7,8,Kyouya,swordmaster,human,M,1,18.0
8,9,Eris,None,god,F,0,NaN


In [7]:
cz.select_from('quest_map').ex()

,quest_id,char_id
0,1,1
1,1,2
2,1,3
3,2,1
4,2,3
5,2,4
6,3,1
7,3,3
8,4,2
9,4,4


In [8]:
cz.select_from('quest').ex()

,id,name,reward
0,1,giant toads,50000
1,2,cabbages,1000000
2,3,explosion practice,0
3,4,repel dullahan,0
4,5,purify the riverhead lake,300000


Kazuma is interested in his potential dating prospects in the world of konosuba, and he does not want to get in trouble with the law, which states that minors are those below 16. He is only interested in women. Classify all female characters at least 16 years of age as 'legal', below as 'jailbait', and those with unknown ages as 'unknown'. Write an sql query that will return the table below:

In [22]:
command = '''
SELECT id as 'character_id',name,race,
    CASE
        WHEN age IS NULL THEN 'unknown'
        WHEN age >= 16 THEN 'legal'
        ELSE 'jailbait'
    END legal_status
FROM konosuba
WHERE sex = 'F'
ORDER BY name
;
'''
pd.read_sql_query(command, engine)

,character_id,name,race,legal_status
0,2,Aqua,god,unknown
1,5,Chris,human,jailbait
2,4,Darkness,human,legal
3,9,Eris,god,unknown
4,3,Megumin,human,jailbait
5,7,Wiz,lich,legal
6,6,Yunyun,human,jailbait


Kazuma wishes to evaluate which of the recent quests he has participated in has been the most rewarding. He rates quests as being highly rewarding if the quest reward / quest participants is above or equal to 50000, medium if below 50000 but above 25000, and low otherwise. Write an sql query that will return the table below:

In [24]:
command = '''
SELECT q.id as 'quest_id', q.name as 'quest',COUNT(*) as 'required_participants',
    CASE
        WHEN AVG(q.reward)/COUNT(*) >= 50000 THEN 'high'
        WHEN AVG(q.reward)/COUNT(*) < 50000 and AVG(q.reward)/COUNT(*) >= 25000 THEN 'medium'
        ELSE 'low'
    END average_quest_reward
FROM konosuba k
INNER JOIN quest_map qm ON qm.char_id = k.id
INNER JOIN quest q ON q.id = qm.quest_id
GROUP BY q.name
ORDER BY AVG(q.reward)/COUNT(*) DESC
;
'''
pd.read_sql_query(command, engine)

,quest_id,quest,required_participants,average_quest_reward
0,2,cabbages,3,high
1,5,purify the riverhead lake,1,high
2,1,giant toads,3,low
3,3,explosion practice,2,low
4,4,repel dullahan,2,low
